In [10]:
import os
import sys

# Handle both notebook and normal script cases
try:
    CURRENT_DIR = os.path.dirname(__file__)
except NameError:
    # __file__ is not defined in notebooks — use the current working directory
    CURRENT_DIR = os.getcwd()

PARENT_DIR = os.path.abspath(os.path.join(CURRENT_DIR, ".."))
if PARENT_DIR not in sys.path:
    sys.path.insert(0, PARENT_DIR)

from DD_FEATURE_EXTRACTOR_09_21_2025 import safe_requests_get

def inspect_url(url: str):
    """Fetch a URL using safe_requests_get and print key response details."""
    if safe_requests_get is None:
        print("safe_requests_get not available; aborting.")
        return
    r = safe_requests_get(url)
    if r is None:
        print("Request failed or returned no response object.")
        return
    print("---------------------------------------------------")
    print("Status:", r.status_code)
    print("Final URL:", r.url)
    print("Content-Type:", r.headers.get("Content-Type"))
    print("Content-Length header:", r.headers.get("Content-Length"))
    body_bytes = r.content or b""
    print("Actual body length (bytes):", len(body_bytes))
    try:
        preview = body_bytes[:2000].decode(r.encoding or "utf-8", errors="replace")
    except Exception:
        preview = "(failed to decode preview)"
    print("---------------------------------------------------")
    print("HTML Preview (first 500 characters):")
    print(preview)
    print("---------------------------------------------------")

if __name__ == "__main__":
    test_url = "https://example.com"
    inspect_url(test_url)


---------------------------------------------------
Status: 200
Final URL: https://example.com/
Content-Type: text/html
Content-Length header: 363
Actual body length (bytes): 513
---------------------------------------------------
HTML Preview (first 500 characters):
<!doctype html><html lang="en"><head><title>Example Domain</title><meta name="viewport" content="width=device-width, initial-scale=1"><style>body{background:#eee;width:60vw;margin:15vh auto;font-family:system-ui,sans-serif}h1{font-size:1.5em}div{opacity:0.8}a:link,a:visited{color:#348}</style><body><div><h1>Example Domain</h1><p>This domain is for use in documentation examples without needing permission. Avoid use in operations.<p><a href="https://iana.org/domains/example">Learn more</a></div></body></html>

---------------------------------------------------
